In [1]:
import syft as sy

assigning id:<UID:1c160ceb-3af9-416e-a265-7a4d41a5b0e7> type:<class 'torch.Tensor'>
setting attr:TensorPointer
setting attr:LinearPointer
setting attr:ndarrayPointer
setting attr:arrayPointer


### Test 1: Send a Message from a VM Client to a VM

In [5]:
bob_vm = sy.VirtualMachine(name="Bob")
bob_vm_client = bob_vm.get_root_client()

setting attr:TensorPointer
setting attr:LinearPointer
setting attr:ndarrayPointer
setting attr:arrayPointer


In [6]:
assert bob_vm.device is None

In [7]:
bob_vm_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm_client))

VirtualMachine:Bob:<UID:a4732401-0204-4210-928e-0604ac89b398>


### Test 2: Send a message from a Device Client to a Device

In [8]:
bob_phone = sy.Device(name="Bob's iPhone")
bob_phone_client = bob_phone.get_client()

setting attr:TensorPointer
setting attr:LinearPointer
setting attr:ndarrayPointer
setting attr:arrayPointer


In [9]:
msg = sy.ReprMessage(address=bob_phone_client)

### Test 2.1: Register a VM on a Device

In [12]:
print(bob_phone_client.routes[0].connection.server.node)
print(bob_phone_client.routes[0].connection.server.node.device)
print(bob_phone_client.device)
print(bob_vm_client)
print(bob_vm_client.vm)
print(bob_vm_client.device) # no device on the vm client yet

Device:Bob's iPhone:<UID:301d5764-3a99-460a-a74a-70fe90514aca>
<SpecificLocation:301d5764-3a99-460a-a74a-70fe90514aca>
<SpecificLocation:301d5764-3a99-460a-a74a-70fe90514aca>
<VirtualMachineClient id:Bob>
<SpecificLocation:a4732401-0204-4210-928e-0604ac89b398>
None


In [13]:
bob_phone_client.register(client=bob_vm_client)

KeyError: "The node <UID:301d5764-3a99-460a-a74a-70fe90514aca> of type <class 'syft.core.node.device.device.Device'> cannot process messages of type <class 'syft.core.common.object.ObjectWithID'> because there is no service running to process it.<class 'syft.core.common.object.ObjectWithID'>"

In [11]:
print(bob_phone.store) # bob_vm_client.id is now stored on the device
assert bob_phone.store[bob_vm_client.id].data == bob_vm_client

{<UID:79e3ad85-f721-426d-825e-d61421bced18>: <Storable:<VirtualMachineClient id:Bob>>}


### Test 2.2 Test that registration updated "address" variable of VM

In [12]:
assert bob_vm.device is not None
assert bob_vm.device.id == bob_phone_client.device.id
print(bob_vm.device)

<SpecificLocation:0ba184d2-776b-44a3-adf7-08dc81447579>


In [13]:
assert bob_vm_client.device is not None
assert bob_vm_client.device.id == bob_phone_client.device.id
print(bob_vm_client.device)

<SpecificLocation:0ba184d2-776b-44a3-adf7-08dc81447579>


### Test 3: Send a message from a Device Client to a VM

In [14]:
print(bob_phone_client.routes[0].connection.server.node)
print(bob_phone_client.routes[0].connection.server.node.device)
print(bob_phone_client.device)
print(bob_vm_client)
print(bob_vm_client.vm)
print(bob_vm_client.device)

Device:Bob's iPhone:<UID:0ba184d2-776b-44a3-adf7-08dc81447579>
<SpecificLocation:0ba184d2-776b-44a3-adf7-08dc81447579>
<SpecificLocation:0ba184d2-776b-44a3-adf7-08dc81447579>
<VirtualMachineClient id:Bob>
<SpecificLocation:79e3ad85-f721-426d-825e-d61421bced18>
<SpecificLocation:0ba184d2-776b-44a3-adf7-08dc81447579>


In [15]:
print(bob_phone_client.id) # phone client id is phone device
print(bob_vm_client.id) # vm client id is vm

<UID:0ba184d2-776b-44a3-adf7-08dc81447579>
<UID:79e3ad85-f721-426d-825e-d61421bced18>


In [16]:
# bobs client has a SoloRoute with a VirtualClientConnection to VirtualServerConnection to bobs_phone
path = [
    bob_phone_client,
    bob_phone_client.routes[0],
    bob_phone_client.routes[0].connection,
    bob_phone_client.routes[0].connection.server,
    bob_phone_client.routes[0].connection.server.node,
    bob_phone_client.routes[0].connection.server.node.message_without_reply_forwarding_service,
    bob_phone_client.routes[0].connection.server.node.store[bob_vm_client.id].data
]
print_path(path)

bob_phone_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm_client))

<DeviceClient:<SpecificLocation:0ba184d2-776b-44a3-adf7-08dc81447579>>
	<SoloRoute:0f908dde-2356-4ee6-b464-a217342e138b>
				Device:Bob's iPhone:<UID:0ba184d2-776b-44a3-adf7-08dc81447579>
						<VirtualMachineClient id:Bob>

the message is not for me... <UID:0ba184d2-776b-44a3-adf7-08dc81447579>
<UID:79e3ad85-f721-426d-825e-d61421bced18>
the message is for me!!!
VirtualMachine:Bob:<UID:79e3ad85-f721-426d-825e-d61421bced18>


### Test 4: Send a message from a Domain Client to a VM

In [17]:
import syft as sy

In [18]:
bob_domain = sy.Domain(name="Bob's Domain")
bob_domain_client = bob_domain.get_client()

setting attr:TensorPointer
setting attr:LinearPointer
setting attr:ndarrayPointer
setting attr:arrayPointer


In [19]:
bob_domain_client.register(client=bob_phone_client)

the message is for me!!!
the message is for me!!!
Updating to Domain:Bob's Domain:<UID:42e9a23c-7356-4ff9-9e08-00497efacfd6> on node Device:Bob's iPhone:<UID:0ba184d2-776b-44a3-adf7-08dc81447579>
child nodes of:Bob's iPhone
	Bob
the message is for me!!!
Updating to Domain:Bob's Domain:<UID:42e9a23c-7356-4ff9-9e08-00497efacfd6> on node VirtualMachine:Bob:<UID:79e3ad85-f721-426d-825e-d61421bced18>
child nodes of:Bob


In [20]:
bob_domain_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm))

the message is not for me... <UID:42e9a23c-7356-4ff9-9e08-00497efacfd6>
<UID:79e3ad85-f721-426d-825e-d61421bced18>
the message is not for me... <UID:0ba184d2-776b-44a3-adf7-08dc81447579>
<UID:79e3ad85-f721-426d-825e-d61421bced18>
the message is for me!!!
VirtualMachine:Bob:<UID:79e3ad85-f721-426d-825e-d61421bced18>


### Test 5: Send a message from Network Client to a VM

In [21]:
bob_network = sy.Network(name="Bob's Network")
bob_network_client = bob_network.get_client()

setting attr:TensorPointer
setting attr:LinearPointer
setting attr:ndarrayPointer
setting attr:arrayPointer


In [22]:
bob_network_client.register(client=bob_domain_client)

the message is for me!!!
the message is for me!!!
Updating to Network:Bob's Network:<UID:c1f00c70-cd73-4f5a-bccf-795ed7978d5d> on node Domain:Bob's Domain:<UID:42e9a23c-7356-4ff9-9e08-00497efacfd6>
child nodes of:Bob's Domain
	Bob's iPhone
the message is for me!!!
Updating to Network:Bob's Network:<UID:c1f00c70-cd73-4f5a-bccf-795ed7978d5d> on node Device:Bob's iPhone:<UID:0ba184d2-776b-44a3-adf7-08dc81447579>
child nodes of:Bob's iPhone
	Bob
the message is for me!!!
Updating to Network:Bob's Network:<UID:c1f00c70-cd73-4f5a-bccf-795ed7978d5d> on node VirtualMachine:Bob:<UID:79e3ad85-f721-426d-825e-d61421bced18>
child nodes of:Bob


In [23]:
bob_network_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm))

the message is not for me... <UID:c1f00c70-cd73-4f5a-bccf-795ed7978d5d>
<UID:79e3ad85-f721-426d-825e-d61421bced18>
the message is not for me... <UID:42e9a23c-7356-4ff9-9e08-00497efacfd6>
<UID:79e3ad85-f721-426d-825e-d61421bced18>
the message is not for me... <UID:0ba184d2-776b-44a3-adf7-08dc81447579>
<UID:79e3ad85-f721-426d-825e-d61421bced18>
the message is for me!!!
VirtualMachine:Bob:<UID:79e3ad85-f721-426d-825e-d61421bced18>


### Test 5: Use VM Client wtih Torch Tensor

In [24]:
print(type(bob_network_client))
print(bob_network_client.routes)
print(bob_network_client.routes[0])
print(bob_network_client.routes[0].connection)

<class 'syft.core.node.network.client.NetworkClient'>
[<SoloRoute:26cde9b1-de53-4088-8b4a-a8ac04654aeb>]
<SoloRoute:26cde9b1-de53-4088-8b4a-a8ac04654aeb>


### Authorized Messages

In [25]:
from syft.core.node.common.service.authorized_service import HelloRootRequest
from nacl.signing import SigningKey, VerifyKey

bobs_signing_key = SigningKey.generate()
bobs_verification_key = bobs_signing_key.verify_key
print(bobs_signing_key) # private dont share with anyone
print(bobs_verification_key) # public give to anyone who needs to verify your messages

b'\x01)\xdb\xc8\x8b$;a=\xb4\x02\xe3\x1d\xac\xc4K\xd4\xcbssx\xad\xdb\xcd\xe4j\xbc\x92\xdfY-\xfb'
b"h\x8c\xaa\x06R\xa4J\\\xcbI\x0cbj\xe5\xca\xd7\x175'hYj:k\xda\x18\x1d]\xbd\xee\x13\x8f"


In [27]:
secure_msg = HelloRootRequest(username="bob", reply_to=bob_phone_client, address=bob_phone)

In [28]:
sm = secure_msg.serialize(to_binary=True)
print(sm)
print(type(sm))

b'{"objType": "syft.core.node.common.service.authorized_service.HelloRootRequest", "content": "{\\"username\\": \\"bob\\", \\"replyTo\\": {\\"hasDomain\\": true, \\"domain\\": {\\"id\\": {\\"value\\": \\"QumiPHNWT/meCABJfvrP1g==\\"}}, \\"hasDevice\\": true, \\"device\\": {\\"id\\": {\\"value\\": \\"C6GE0ndrRKOt9wjcgUR1eQ==\\"}}}, \\"address\\": {\\"hasNetwork\\": true, \\"network\\": {\\"id\\": {\\"value\\": \\"wfAMcM1zT1q8z3le15eNXQ==\\"}}, \\"hasDomain\\": true, \\"domain\\": {\\"id\\": {\\"value\\": \\"QumiPHNWT/meCABJfvrP1g==\\"}}, \\"hasDevice\\": true, \\"device\\": {\\"id\\": {\\"value\\": \\"C6GE0ndrRKOt9wjcgUR1eQ==\\"}}}, \\"msgId\\": {\\"value\\": \\"kwDxKbGlSXuf7JItR/v9Ww==\\"}}"}'
<class 'bytes'>


In [29]:
dm = sy.deserialize(blob=sm, from_binary=True)
print(dm)
print(type(dm))

<HelloRootRequest:9300f129-b1a5-497b-9fec-922d47fbfd5b>
<class 'syft.core.node.common.service.authorized_service.HelloRootRequest'>


In [30]:
signed_msg = secure_msg.sign_message(signing_key=bobs_signing_key)
signed_msg.address = secure_msg.address
print(signed_msg.address)

Device:Bob's iPhone:<UID:0ba184d2-776b-44a3-adf7-08dc81447579>


In [31]:
sig_hello = bob_phone_client.send_signed_msg_with_reply(msg=signed_msg)
print(sig_hello)

hello = sig_hello.inner_message()
print(hello.message)

Device:Bob's iPhone:<UID:0ba184d2-776b-44a3-adf7-08dc81447579> <HelloRootRequest:9300f129-b1a5-497b-9fec-922d47fbfd5b>
Hello bob


In [32]:
# make a tampered message
tampered_msg = secure_msg.sign_message(signing_key=bobs_signing_key)
tampered_msg.address = secure_msg.address
tampered_msg.signature += b"a"

In [33]:
sig_hello = bob_phone_client.send_signed_msg_with_reply(msg=tampered_msg)
print(sig_hello)

hello = sig_hello.inner_message(allow_invalid=True)
print(hello.message)

Device:Bob's iPhone:<UID:0ba184d2-776b-44a3-adf7-08dc81447579> <HelloRootRequest:9300f129-b1a5-497b-9fec-922d47fbfd5b>
Unauthorized. Go away bob!


# Auth signing any message

In [1]:
import syft as sy
from syft.core.node.common.service.repr_service import ReprMessage

/Users/atrask/opt/anaconda3/envs/syft/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py:102: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


assigning id:<UID:d3bfda2b-33ac-4afa-a25b-a69416db4a19> type:<class 'torch.Tensor'>
setting attr:TensorPointer
setting attr:LinearPointer
setting attr:ParameterPointer


In [2]:
bob = sy.VirtualMachine(name="bob")
bobc = bob.get_root_client()

alice = sy.VirtualMachine(name="alice")
alicec = bob.get_client()

> Creating 🍰 bob (VirtualMachine@<UID:🚰🚃>)
setting attr:TensorPointer
setting attr:LinearPointer
setting attr:ParameterPointer
> Creating 📡 [🍰] bob Client (VirtualMachineClient@<UID:🚰🚃>)
> Creating 🍰 alice (VirtualMachine@<UID:🚰🚃>)
setting attr:TensorPointer
setting attr:LinearPointer
setting attr:ParameterPointer
> Creating 📡 [🍰] bob Client (VirtualMachineClient@<UID:🚰🚃>)


In [3]:
from syft.core.io.address import Address

In [4]:
import torch as th
x = th.tensor([1,2,3,4])

assigning id:<UID:b9d772d9-55ba-4ac6-99ba-6bbb6c69ba60> type:<class 'torch.Tensor'>


In [5]:
ptr = x.send(bobc)

> Creating ✉️  (SaveObjectAction)
> 📡 [🍰] bob Client (VirtualMachineClient@<UID:🚰🚃>) Signing ✉️  (SaveObjectAction) with 🜨
> Signing with 🜨
Wrapped tensor with id:<UID:b9d772d9-55ba-4ac6-99ba-6bbb6c69ba60>
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)
> Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [🍰] bob Client (VirtualMachineClient@<UID:🚰🚃>) ➡️  💠 [🍰] bob Client (Address
> Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
> Received ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 bob (VirtualMachine@<UID:🚰🚃>)
> Processing 📨 ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 bob (VirtualMachine@<UID:🚰🚃>)
> Recipient Found ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)@<UID:🚰🚃> == 🍰 bob (VirtualMachine@<UID:🚰🚃>)
assigning id:<UID:57750a08-cd11-4491-b2fc-502bb4d00856> type:<class 'torch.Tensor'>
> Creating ✉️  (SaveObjectAction)


In [6]:
fake_bob = bob.get_client()

setting attr:TensorPointer
setting attr:LinearPointer
setting attr:ParameterPointer
> Creating 📡 [🍰] bob Client (VirtualMachineClient@<UID:🚰🚃>)


In [7]:
ptr.location = fake_bob

In [8]:
ptr.get()

> Creating ✉️  (GetObjectAction)
> 📡 [🍰] bob Client (VirtualMachineClient@<UID:🚰🚃>) Signing ✉️  (GetObjectAction) with 🜧
> Signing with 🜧
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithReply)
> Processing 📨 ✉️ 🔏 (SignedImmediateSyftMessageWithReply) @ 🍰 bob (VirtualMachine@<UID:🚰🚃>)
> Recipient Found ✉️ 🔏 (SignedImmediateSyftMessageWithReply)@<UID:🚰🚃> == 🍰 bob (VirtualMachine@<UID:🚰🚃>)
> Creating ✉️  (GetObjectAction)


Exception: You do not have permission to .get() this tensor. Please submit a request.

In [9]:
import pytest

bob = sy.VirtualMachine(name="bob")
root_bob = bob.get_root_client()
guest_bob = bob.get_client()

import torch as th
x = th.tensor([1, 2, 3, 4])

# root user of Bob's machine sends a tensor
ptr = x.send(root_bob)

# guest bob creates a pointer to that object (assuming he could guess/inpher the ID)
ptr.location = guest_bob

# this should trigger an exception
with pytest.raises(Exception):
    ptr.get()

x = th.tensor([1, 2, 3, 4])

# root user of Bob's machine sends a tensor
ptr = x.send(root_bob)

# but if root bob asks for it it should be fine
x2 = ptr.get()

assert (x == x2).all()

> Creating 🍰 bob (VirtualMachine@<UID:🚰🚃>)
setting attr:TensorPointer
setting attr:LinearPointer
setting attr:ParameterPointer
> Creating 📡 [🍰] bob Client (VirtualMachineClient@<UID:🚰🚃>)
setting attr:TensorPointer
setting attr:LinearPointer
setting attr:ParameterPointer
> Creating 📡 [🍰] bob Client (VirtualMachineClient@<UID:🚰🚃>)
assigning id:<UID:52c02623-fa04-4ac2-9f6a-7051f84df1af> type:<class 'torch.Tensor'>
> Creating ✉️  (SaveObjectAction)
> 📡 [🍰] bob Client (VirtualMachineClient@<UID:🚰🚃>) Signing ✉️  (SaveObjectAction) with 🞁
> Signing with 🞁
Wrapped tensor with id:<UID:52c02623-fa04-4ac2-9f6a-7051f84df1af>
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)
> Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [🍰] bob Client (VirtualMachineClient@<UID:🚰🚃>) ➡️  💠 [🍰] bob Client (Address
> Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
> Received ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 bob (VirtualMachine@<UID:🚰🚃>)
> Processing 

In [6]:
msg = ReprMessage(address=bobc).sign(signing_key=bobc.signing_key)

In [4]:
bobc.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bobc))

VirtualMachine:bob:<UID:542dc099-bbf7-4ca2-bb59-7bda22d30b6e>


In [5]:
alicec.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=alicec))

AssertionError: 

In [34]:
import syft as sy
alice = sy.VirtualMachine(name="alice")
alice_client = alice.g
et_client()

setting attr:TensorPointer
setting attr:LinearPointer
setting attr:ndarrayPointer
setting attr:arrayPointer


In [35]:
import torch as th

In [36]:
x = th.tensor([1,2,3,4])

blob = x.serialize()

x2 = sy.deserialize(blob=blob)

assert (x == x2).all()

assigning id:<UID:a9e8eaf9-c2ac-4418-bf22-8e0652c2184d> type:<class 'torch.Tensor'>
Wrapped tensor with id:<UID:a9e8eaf9-c2ac-4418-bf22-8e0652c2184d>
assigning id:<UID:5ebe8095-974a-4fbd-b553-316701950218> type:<class 'torch.Tensor'>


In [37]:
x.id

<UID:a9e8eaf9-c2ac-4418-bf22-8e0652c2184d>

In [38]:
xp = x.send(alice_client)

the message is for me!!!


In [39]:
xp.id_at_location

<UID:a9e8eaf9-c2ac-4418-bf22-8e0652c2184d>

In [40]:
y = xp + xp

id at location:<UID:821f4e5c-0e82-4f01-b2de-a09465064fdf>
the message is for me!!!


In [41]:
y.get()

tensor([2, 4, 6, 8])

In [42]:
alice.store

{<UID:a9e8eaf9-c2ac-4418-bf22-8e0652c2184d>: <Storable:tensor([1, 2, 3, 4])>}

In [43]:
del xp

In [44]:
alice.store

{<UID:a9e8eaf9-c2ac-4418-bf22-8e0652c2184d>: <Storable:tensor([1, 2, 3, 4])>}

In [45]:
ptr = alice_client.torch.zeros(10,10)

call torch.zeros on client <VirtualMachineClient id:alice>
the message is for me!!!


In [46]:
import numpy as np
import syft as sy

In [47]:
x = np.array([1,2,3,4]).send(alice_client)

assigning id:<UID:e227b792-ace0-4920-8c25-b9cb92e22b23> type:<class 'syft.lib.generic.ObjectConstructor.install_id_attribute.<locals>.OriginalConstructorSubclass'>
the message is for me!!!


In [48]:
x.get()

OriginalConstructorSubclass([[[[ 2.00000000e+000,  2.00000000e+000,
                                 1.05895010e-153,  2.13890927e+161],
                               [ 2.34601258e-153,  2.63570224e+092,
                                 6.29201311e+233,  6.09343068e-013],
                               [ 4.94277337e+160,  4.82562168e+276,
                                 1.11319014e+171,  2.64685963e+180]],

                              [[ 7.68755624e+170,  2.51719506e+180,
                                 4.65682141e+164,  4.97749602e+151],
                               [ 2.33520274e+030,  3.43511257e+228,
                                 1.02584502e+200,  3.45211702e+175],
                               [ 7.06357831e-308,  0.00000000e+000,
                                 2.00000000e+000, -1.73059638e-077]]]])

In [49]:
x = np.array([1,2,3,4])

assigning id:<UID:425d7182-3fd7-4563-af64-c873a1d584fe> type:<class 'syft.lib.generic.ObjectConstructor.install_id_attribute.<locals>.OriginalConstructorSubclass'>


In [50]:
x.serialize()

Wrapped tensor with id:<UID:425d7182-3fd7-4563-af64-c873a1d584fe>


id {
  value: "B]q\202?\327Ec\257d\310s\241\325\204\376"
}
obj_type: "syft.lib.numpy.tensor.NumpyTensorWrapper"
data {
  type_url: "type.googleapis.com/syft.lib.numpy.TensorProto"
  value: "\n\004\001\002\003\004\020\r:\300\001\000\000\000\000\000\000\000@\000\000\000\000\000\000\000@\r\000size, *, DimnameList? names, ScalarType dtype=None, Layout layout=torch.strided, Device device=None, bool pin_memory=False, bool requires_grad=False)\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000@w\242\222l\371\007\000\260"
}

In [51]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)

In [52]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

In [53]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [54]:
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    password = Column(String)

    def __repr__(self):
        return f'User {self.name}'

In [55]:
Base.metadata.create_all(engine)

2020-08-12 14:33:52,985 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-08-12 14:33:52,997 INFO sqlalchemy.engine.base.Engine ()
2020-08-12 14:33:53,003 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-08-12 14:33:53,006 INFO sqlalchemy.engine.base.Engine ()
2020-08-12 14:33:53,011 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-08-12 14:33:53,013 INFO sqlalchemy.engine.base.Engine ()
2020-08-12 14:33:53,026 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2020-08-12 14:33:53,028 INFO sqlalchemy.engine.base.Engine ()
2020-08-12 14:33:53,033 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2020-08-12 14:33:53,035 INFO sqlalchemy.engine.base.Engine ()
2020-08-12 14:33:53,038 INFO sqlalchemy.engine.base.Engine COMMIT


In [56]:
user = User(name='John Snow', password='johnspassword')
session.add(user)

print(user.id)  # None

None


In [57]:
session.commit()

2020-08-12 14:33:53,091 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-12 14:33:53,096 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, password) VALUES (?, ?)
2020-08-12 14:33:53,098 INFO sqlalchemy.engine.base.Engine ('John Snow', 'johnspassword')
2020-08-12 14:33:53,101 INFO sqlalchemy.engine.base.Engine COMMIT


In [58]:
query = session.query(User).filter_by(name='John Snow')

In [59]:
query.count()

2020-08-12 14:33:53,155 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-12 14:33:53,160 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.password AS users_password 
FROM users 
WHERE users.name = ?) AS anon_1
2020-08-12 14:33:53,162 INFO sqlalchemy.engine.base.Engine ('John Snow',)


1